# Детекторы ключевых точек (SIFT, SURF, ORB)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os

In [ ]:
# Загрузка тестового изображения
img_path = 'sample.jpg'

if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(f"✅ Изображение загружено: {img_path}")
else:
    print("📝 Создаём тестовое изображение с текстурированными объектами...")

    # Создаём изображение с различными объектами для детекции ключевых точек
    img = np.ones((500, 700, 3), dtype=np.uint8) * 240  # Светлый фон

    # Добавляем текстурированные области
    # Прямоугольник с градиентом
    for i in range(100, 200):
        for j in range(50, 200):
            img[i, j] = [100 + (i-100), 150 + (j-50)//2, 200]

    # Шахматная доска (много углов)
    square_size = 30
    for i in range(8):
        for j in range(8):
            if (i + j) % 2 == 0:
                y1, y2 = 50 + i*square_size, 50 + (i+1)*square_size
                x1, x2 = 300 + j*square_size, 300 + (j+1)*square_size
                img[y1:y2, x1:x2] = [50, 50, 50]

    # Круги разного размера
    cv2.circle(img, (150, 350), 40, (100, 100, 180), -1)
    cv2.circle(img, (150, 350), 30, (200, 200, 220), -1)
    cv2.circle(img, (250, 350), 25, (180, 100, 100), -1)

    # Линии под разными углами
    cv2.line(img, (400, 300), (550, 400), (150, 150, 150), 3)
    cv2.line(img, (400, 400), (550, 300), (150, 150, 150), 3)

    # Текст (много особенностей)
    cv2.putText(img, 'SIFT', (450, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (80, 80, 80), 3)

    # Добавляем небольшой шум для реалистичности
    noise = np.random.randint(-10, 10, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("✅ Синтетическое изображение создано")

# Преобразуем в grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Отображаем исходное изображение
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Исходное изображение (RGB)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_gray, cmap='gray')
plt.title('Grayscale версия')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Инициализация и сравнение детекторов
# ===============================

# Создаём детекторы
print("🔧 Инициализация детекторов...")

# SIFT
try:
    sift = cv2.SIFT_create()
    print("✅ SIFT инициализирован")
except:
    print("❌ SIFT недоступен (требуется opencv-contrib-python)")
    sift = None

# SURF (может отсутствовать в некоторых версиях OpenCV)
try:
    surf = cv2.xfeatures2d.SURF_create(400)  # порог 400
    print("✅ SURF инициализирован")
except:
    print("❌ SURF недоступен (требуется opencv-contrib-python с включённым xfeatures2d)")
    surf = None

# ORB
try:
    orb = cv2.ORB_create(nfeatures=1000)  # максимум 1000 точек
    print("✅ ORB инициализирован")
except:
    print("❌ ORB недоступен")
    orb = None

# Детектируем ключевые точки и вычисляем дескрипторы
detectors = {}

if sift is not None:
    start_time = time.time()
    kp_sift, des_sift = sift.detectAndCompute(img_gray, None)
    time_sift = time.time() - start_time
    detectors['SIFT'] = {'kp': kp_sift, 'des': des_sift, 'time': time_sift}
    print(f"📊 SIFT: {len(kp_sift)} ключевых точек за {time_sift:.3f} сек")

if surf is not None:
    start_time = time.time()
    kp_surf, des_surf = surf.detectAndCompute(img_gray, None)
    time_surf = time.time() - start_time
    detectors['SURF'] = {'kp': kp_surf, 'des': des_surf, 'time': time_surf}
    print(f"📊 SURF: {len(kp_surf)} ключевых точек за {time_surf:.3f} сек")

if orb is not None:
    start_time = time.time()
    kp_orb, des_orb = orb.detectAndCompute(img_gray, None)
    time_orb = time.time() - start_time
    detectors['ORB'] = {'kp': kp_orb, 'des': des_orb, 'time': time_orb}
    print(f"📊 ORB: {len(kp_orb)} ключевых точек за {time_orb:.3f} сек")

# Визуализируем ключевые точки
if detectors:
    num_detectors = len(detectors)
    fig, axes = plt.subplots(1, num_detectors + 1, figsize=(5*(num_detectors+1), 5))

    # Исходное изображение
    axes[0].imshow(img_gray, cmap='gray')
    axes[0].set_title('Исходное изображение')
    axes[0].axis('off')

    # Результаты детекторов
    for i, (name, data) in enumerate(detectors.items(), 1):
        img_kp = cv2.drawKeypoints(
            img_gray,
            data['kp'],
            None,
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS  # Рисуем с размером и ориентацией
        )
        img_kp_rgb = cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB)

        axes[i].imshow(img_kp_rgb)
        axes[i].set_title(f'{name}\n{len(data["kp"])} точек, {data["time"]:.3f} сек')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Анализ свойств ключевых точек
# ===============================

def analyze_keypoints(keypoints, name):
    """Анализирует свойства ключевых точек"""

    if len(keypoints) == 0:
        print(f"⚠️  {name}: нет ключевых точек для анализа")
        return

    # Извлекаем свойства
    positions = np.array([kp.pt for kp in keypoints])
    sizes = np.array([kp.size for kp in keypoints])
    angles = np.array([kp.angle for kp in keypoints])
    responses = np.array([kp.response for kp in keypoints])

    print(f"\n🔍 Анализ {name}:")
    print(f"   Количество точек: {len(keypoints)}")
    print(f"   Размер (scale):")
    print(f"      Мин: {sizes.min():.2f}, Макс: {sizes.max():.2f}, Средний: {sizes.mean():.2f}")
    print(f"   Ориентация:")
    print(f"      Мин: {angles.min():.2f}°, Макс: {angles.max():.2f}°")
    print(f"   Response (сила отклика):")
    print(f"      Мин: {responses.min():.6f}, Макс: {responses.max():.6f}, Средний: {responses.mean():.6f}")

    return positions, sizes, angles, responses

# Анализируем каждый детектор
keypoint_properties = {}

for name, data in detectors.items():
    props = analyze_keypoints(data['kp'], name)
    if props is not None:
        keypoint_properties[name] = props

# Визуализируем распределения
if keypoint_properties:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Распределение размеров
    axes[0, 0].set_title('Распределение размеров (scale)')
    for name, (pos, sizes, angles, resp) in keypoint_properties.items():
        axes[0, 0].hist(sizes, bins=30, alpha=0.5, label=name)
    axes[0, 0].set_xlabel('Размер')
    axes[0, 0].set_ylabel('Количество')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Распределение ориентаций
    axes[0, 1].set_title('Распределение ориентаций')
    for name, (pos, sizes, angles, resp) in keypoint_properties.items():
        axes[0, 1].hist(angles, bins=36, alpha=0.5, label=name)
    axes[0, 1].set_xlabel('Угол (градусы)')
    axes[0, 1].set_ylabel('Количество')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Распределение response
    axes[1, 0].set_title('Распределение силы отклика')
    for name, (pos, sizes, angles, resp) in keypoint_properties.items():
        axes[1, 0].hist(resp, bins=30, alpha=0.5, label=name)
    axes[1, 0].set_xlabel('Response')
    axes[1, 0].set_ylabel('Количество')
    axes[1, 0].set_yscale('log')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Spatial distribution (карта плотности)
    axes[1, 1].set_title('Пространственное распределение')
    for name, (pos, sizes, angles, resp) in keypoint_properties.items():
        axes[1, 1].scatter(pos[:, 0], pos[:, 1], alpha=0.3, s=10, label=name)
    axes[1, 1].set_xlim(0, img_gray.shape[1])
    axes[1, 1].set_ylim(img_gray.shape[0], 0)
    axes[1, 1].set_xlabel('X координата')
    axes[1, 1].set_ylabel('Y координата')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Анализ дескрипторов
# ===============================

for name, data in detectors.items():
    des = data['des']

    if des is None:
        print(f"⚠️  {name}: дескрипторы отсутствуют")
        continue

    print(f"\n🔍 Дескрипторы {name}:")
    print(f"   Форма: {des.shape} (количество × размерность)")
    print(f"   Тип данных: {des.dtype}")
    print(f"   Размер в памяти: {des.nbytes / 1024:.2f} КБ")

    if name in ['SIFT', 'SURF']:
        # Для float дескрипторов
        print(f"   Диапазон значений: [{des.min():.2f}, {des.max():.2f}]")
        print(f"   Среднее значение: {des.mean():.2f}")
        print(f"   Стд. отклонение: {des.std():.2f}")
    else:
        # Для бинарных дескрипторов (ORB)
        print(f"   Бинарный дескриптор: {des.shape[1] * 8} бит")

# Визуализируем примеры дескрипторов
if detectors:
    fig, axes = plt.subplots(1, len(detectors), figsize=(6*len(detectors), 4))

    if len(detectors) == 1:
        axes = [axes]

    for i, (name, data) in enumerate(detectors.items()):
        des = data['des']

        if des is not None and len(des) > 0:
            # Показываем первые 10 дескрипторов
            sample = des[:min(10, len(des))]

            axes[i].imshow(sample, cmap='viridis', aspect='auto')
            axes[i].set_title(f'{name} дескрипторы\n(первые {len(sample)} точек)')
            axes[i].set_xlabel('Размерность')
            axes[i].set_ylabel('Индекс ключевой точки')
            plt.colorbar(axes[i].images[0], ax=axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Тестирование инвариантности
# ===============================

def create_transformed_images(image):
    """Создаёт набор трансформированных изображений"""

    height, width = image.shape[:2]
    transforms = {}

    # Масштабирование
    transforms['scale_50%'] = cv2.resize(image, None, fx=0.5, fy=0.5)
    transforms['scale_150%'] = cv2.resize(image, None, fx=1.5, fy=1.5)
    transforms['scale_200%'] = cv2.resize(image, None, fx=2.0, fy=2.0)

    # Поворот
    center = (width // 2, height // 2)

    M30 = cv2.getRotationMatrix2D(center, 30, 1.0)
    transforms['rotate_30°'] = cv2.warpAffine(image, M30, (width, height))

    M90 = cv2.getRotationMatrix2D(center, 90, 1.0)
    transforms['rotate_90°'] = cv2.warpAffine(image, M90, (width, height))

    M180 = cv2.getRotationMatrix2D(center, 180, 1.0)
    transforms['rotate_180°'] = cv2.warpAffine(image, M180, (width, height))

    # Изменение яркости
    transforms['bright_+50'] = np.clip(image.astype(np.int16) + 50, 0, 255).astype(np.uint8)
    transforms['bright_-50'] = np.clip(image.astype(np.int16) - 50, 0, 255).astype(np.uint8)

    # Размытие
    transforms['blur'] = cv2.GaussianBlur(image, (7, 7), 2)

    return transforms

# Создаём трансформации
print("🔄 Создаём трансформированные версии изображения...")
transformed_images = create_transformed_images(img_gray)

print(f"✅ Создано {len(transformed_images)} трансформаций")

# Детектируем ключевые точки на всех трансформациях
print("\n🔍 Детектируем ключевые точки на трансформациях...")

invariance_results = {}

for detector_name, detector_data in detectors.items():
    print(f"\n📊 Тестируем {detector_name}:")

    results = {'original': len(detector_data['kp'])}

    # Получаем детектор
    if detector_name == 'SIFT':
        detector = sift
    elif detector_name == 'SURF':
        detector = surf
    elif detector_name == 'ORB':
        detector = orb

    for transform_name, transformed_img in transformed_images.items():
        kp, _ = detector.detectAndCompute(transformed_img, None)
        results[transform_name] = len(kp)
        print(f"   {transform_name}: {len(kp)} точек")

    invariance_results[detector_name] = results

# Визуализируем результаты тестирования инвариантности
if invariance_results:
    # Подготовка данных для графика
    transform_names = list(next(iter(invariance_results.values())).keys())

    fig, ax = plt.subplots(figsize=(14, 6))

    x = np.arange(len(transform_names))
    width = 0.25

    for i, (detector_name, results) in enumerate(invariance_results.items()):
        counts = [results[t] for t in transform_names]
        offset = (i - len(invariance_results)/2 + 0.5) * width
        ax.bar(x + offset, counts, width, label=detector_name, alpha=0.8)

    ax.set_xlabel('Тип трансформации')
    ax.set_ylabel('Количество ключевых точек')
    ax.set_title('Сравнение детекторов: инвариантность к трансформациям')
    ax.set_xticks(x)
    ax.set_xticklabels(transform_names, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Вычисляем стабильность (отношение к исходному количеству)
print("\n📈 Стабильность детекторов (% от исходного количества):")

for detector_name, results in invariance_results.items():
    original_count = results['original']
    print(f"\n{detector_name}:")

    for transform_name, count in results.items():
        if transform_name != 'original':
            stability = (count / original_count) * 100 if original_count > 0 else 0
            print(f"   {transform_name}: {stability:.1f}%")

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Визуализация пирамиды масштабов (для SIFT)
# ===============================

if sift is not None:
    def build_gaussian_pyramid(image, n_octaves=4, scales_per_octave=5):
        """Строит гауссову пирамиду для визуализации"""

        pyramid = []
        sigma = 1.6
        k = 2**(1/scales_per_octave)

        for octave in range(n_octaves):
            octave_images = []

            # Изменяем размер для новой октавы
            if octave == 0:
                current_img = image.copy()
            else:
                current_img = cv2.resize(
                    pyramid[octave-1][0],
                    None,
                    fx=0.5,
                    fy=0.5,
                    interpolation=cv2.INTER_LINEAR
                )

            # Генерируем масштабы внутри октавы
            for scale in range(scales_per_octave):
                current_sigma = sigma * (k ** scale)
                blurred = cv2.GaussianBlur(
                    current_img,
                    (0, 0),
                    current_sigma
                )
                octave_images.append(blurred)

            pyramid.append(octave_images)

        return pyramid

    print("🔄 Строим пирамиду масштабов...")
    pyramid = build_gaussian_pyramid(img_gray, n_octaves=3, scales_per_octave=4)

    # Визуализируем пирамиду
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))

    for octave_idx, octave in enumerate(pyramid):
        for scale_idx, img_scale in enumerate(octave):
            axes[octave_idx, scale_idx].imshow(img_scale, cmap='gray')
            axes[octave_idx, scale_idx].set_title(
                f'Октава {octave_idx+1}, Масштаб {scale_idx+1}\n'
                f'Размер: {img_scale.shape[1]}×{img_scale.shape[0]}'
            )
            axes[octave_idx, scale_idx].axis('off')

    plt.suptitle('Гауссова пирамида масштабов (как в SIFT)', fontsize=14)
    plt.tight_layout()
    plt.show()

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте упрощённую версию FAST детектора углов (проверка 16 точек на окружности)

2. Сравните все детекторы на 5 различных типах изображений:
- Текстуры (ткань, дерево)
- Архитектура (здания)
- Природа (пейзажи)
- Объекты (предметы на столе)
- Лица (портреты)

3. Исследуйте экстремальные трансформации:
- Масштаб 10% и 500%
- Сильное размытие (kernel 21x21)
- Комбинированные трансформации